In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('2024-04-30T15-00_20-00_RangelineSMedicalDr.csv', parse_dates=['capture_ts'], index_col='capture_ts', date_format='ISO8601').drop(columns=['detection_id'])

: 

### First, reuse the code from the previous exercise to plot the object centers on the camera image

In [ ]:
CLASSNAME_BY_ID = {0: 'person', 1: 'bicycle', 2: 'car'}

# Calculate the center of each bounding box and add two new columns to the DataFrame: 'center_x' and 'center_y'
df['center_x'] = (df['min_x'] + df['max_x']) / 2
df['center_y'] = (df['min_y'] + df['max_y']) / 2

# Plot the center of each bounding box over time (use different colors and correct labels)
plt.figure(figsize=(16, 9))
for class_id in df['class_id'].unique():
    class_data = df[df['class_id'] == class_id]
    plt.scatter(class_data['center_x'], class_data['center_y'], label=CLASSNAME_BY_ID[class_id], s=.01)
plt.xlabel('center_x')
plt.ylabel('center_y')
plt.legend(loc='upper right')

# Make this resemble the original camera image by setting the origin to the top left and scaling the axis to a 16:9 aspect ratio
plt.gca().invert_yaxis()
plt.gca().set_aspect(0.5625, adjustable='box')

### Now, use the geo coordinates from the data to draw the points

In [ ]:
# Draw the object centers by geo location (latitude and longitude)
# To make it fit a bit nicer, you can set the upper and lower axis limits to the 1st and 99th percentile of the data
plt.figure(figsize=(10, 15))
plt.scatter(df['longitude'], df['latitude'], s=.01)
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.xlim(df['longitude'].quantile(.01), df['longitude'].quantile(.99))
plt.ylim(df['latitude'].quantile(.01), df['latitude'].quantile(.99))
plt.gca().set_aspect(1, adjustable='box')


### Calculate speeds

In [ ]:
# You can assume that one degree of latitude is approximately 111034.132m and one degree of longitude is approximately 85424.462m at 39.9754° N (see calculator http://www.csgnetwork.com/degreelenllavcalc.html)
# With the difference in time and location between any two consecutive detections of the same object, you can calculate the object speeds!

# Sort by object_id and capture_ts
df_sorted = df.sort_values(by=['object_id', 'capture_ts'])

# Calculate the time difference between consecutive detections of the same object (shift the capture_ts column by one and subtract it from the original)
df_sorted['time_diff'] = df_sorted.index.diff()

# Add 'valid_diff' column to the DataFrame to indicate if the time difference is valid (i.e. the object_id is the same as the previous row)
df_sorted['valid_diff'] = df_sorted['object_id'].eq(df_sorted['object_id'].shift())

# Calculate the distance between consecutive detections of the same object (use euclidean distance)
# Calculate the speed by dividing the distance by the time difference
# Add 'speed' column to the DataFrame
df_sorted['distance'] = ((df_sorted['longitude'].diff() * 85424.462)**2 + (df_sorted['latitude'].diff() * 111034.132)**2 )**.5

# Remove the rows where 'valid_diff' is False
df_sorted = df_sorted[df_sorted['valid_diff']]

# Calculate the speed in km/h and add it to the DataFrame
df_sorted['speed'] = (df_sorted['distance'] / df_sorted['time_diff'].dt.total_seconds()) * 3.6

In [ ]:
# Plot a histogram of the object speeds
# Remove the outliers by setting the upper limit to the 99th percentile of the data
df_sorted[df_sorted['speed'] < df_sorted['speed'].quantile(.99)]['speed'].plot.hist(bins=100)



### Plot object locations again with speed encoded as color

In [ ]:
# Draw the object centers by geo location (latitude and longitude)
# To make it fit a bit nicer, you can set the upper and lower axis limits to the 1st and 99th percentile of the data
df_speeds = df_sorted[df_sorted['speed'] < df_sorted['speed'].quantile(.99)]
plt.figure(figsize=(15, 15))
plt.scatter(df_speeds['longitude'], df_speeds['latitude'], c=df_speeds['speed'], s=.01)
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.xlim(df_speeds['longitude'].quantile(.01), df_speeds['longitude'].quantile(.99))
plt.ylim(df_speeds['latitude'].quantile(.01), df_speeds['latitude'].quantile(.99))
plt.colorbar(location='bottom', label='speed (km/h)')
plt.gca().set_aspect(1, adjustable='box')